## Let's explore the Labware object

Let's start by importing the `Labware` module

In [5]:
from science_jubilee.labware.Labware import Labware

### 1. Load a labware file

Labware files follow the same structure as the *Opentrons Labware Definition*. In fact, it is possible to use the [Opentrons Library files](https://github.com/Opentrons/opentrons/tree/5f8e7f60ba80d3a36f42e029f14c216f3f7390fa/shared-data/labware/definitions/2) directly or create a custom one using their [*Custom Labware Creator*](https://labware.opentrons.com/create). All labware definition files can either be stored locally in any folder of your computer or for easier reference can be added to the `labware/labware_definition/` folder in the repository. 

**Naming Convention**: we decided to follow the same naming strategy as Opentrons. In this case, the standard is `<brand_numberOfWells_labwareType_wellVolume>.json` (e.g., *biorad_96_wellplate_200ul_pcr.json*)

*Note*: If you choose to use a file already present in the [Opentrons Library ](https://github.com/Opentrons/opentrons/tree/5f8e7f60ba80d3a36f42e029f14c216f3f7390fa/shared-data/labware/definitions/2), navigate to its folder, download it and make sure to change the file name to reflect the actualy labware (i.e., change it to the folder name)

In [6]:
#define the file name ( .json can be omitted here)
labware_file = '20mlscintillation_12_wellplate_18000ul'
lab = Labware(labware_file)
lab

wellPlate: 20mlscintillation_12_wellplate_18000ul

If you only indicate the labware_filename, the `path` of the file will taken as the `labware/labware_definition`. You can also load a file from a different path, by just adding it as a keywork argument.

In [ ]:
labware_path = 'path/to/my/labware/file'
lab = Labware(labware_file, path = labware_path)

### 2. Well Object

A Labware is defined as a set of individual `Well`s. Each of them is a `dataclass` that stores information on its location, geometry, and more. Let's explore it here briefly.

*A.* First of all, you can call each `Well` based on its *name* (i.e., *RowLetter ColumnNumber*), or by its *index* (i.e., 0,1,...n )

In [7]:
lab['C3']

Well(name='C3', depth=53, totalLiquidVolume=18000, shape='circular', diameter=27.85, xDimension=None, yDimension=None, x=77.47, y=15.03, z=7.5, offset=None, slot=None)

In [8]:
lab[10]

Well(name='B4', depth=53, totalLiquidVolume=18000, shape='circular', diameter=27.85, xDimension=None, yDimension=None, x=105.3, y=42.86, z=7.5, offset=None, slot=None)

*B.* Each attribute of the `Well` can be easily accessed

In [9]:
lab['C3'].x

77.47

In [10]:
lab['C3'].depth # the total depth from the well's top

53

*C.* There are also some properties of the `Well`

In [13]:
lab[10].top_ # the actual top of the well

60.5

However, you can also indicate a `z` position with resposect to the top or bottom of the well.
The `x` and `y` coordinates are the same, this command will change the `z` coordinate and return a Location(Point, Well) object

In [14]:
lab[10].bottom(+5) 

Location(point=(105.3, 42.86, 12.5), labware=Well(name='B4', depth=53, totalLiquidVolume=18000, shape='circular', diameter=27.85, xDimension=None, yDimension=None, x=105.3, y=42.86, z=7.5, offset=None, slot=None))

### 3. Well Order

It is possible to load a labware and define the `Well` ordering you prefer (i.e., by rows or by columns). 
To do so, you can either specify it during the instatiation of the `Labware` object using the keywork argument `order`.

*Note*: the default is ordering by *rows*

In [15]:
# check the current well ordering of the labware just loaded
lab.wells.keys()

dict_keys(['A1', 'B1', 'C1', 'A2', 'B2', 'C2', 'A3', 'B3', 'C3', 'A4', 'B4', 'C4'])

The labdware loaded without indicating any order preference, does in fact show a row-wise list of the well. 

Now to change that, we can just call the method `withWellOrder()` and indicate columns as our preferred `Well` arrangement.

In [16]:
lab.withWellOrder('columns')
lab.wells.keys()

dict_keys(['A1', 'A2', 'A3', 'A4', 'B1', 'B2', 'B3', 'B4', 'C1', 'C2', 'C3', 'C4'])

It is also possible to do so while loading the labware configuration file.

In [17]:
new_labware = Labware(labware_file, order='cols')
new_labware.wells.keys()

dict_keys(['A1', 'A2', 'A3', 'A4', 'B1', 'B2', 'B3', 'B4', 'C1', 'C2', 'C3', 'C4'])

### 4. Offset

It is possible to offset all the well coordinates by an arbitrary amount in all 3-dimensions. 
This is actually how the labware is *loaded* onto the deck object. Once you select a `deck.slots` to host your labware, its coordinates will be adjustated to take into account the slot corner. It is also possible to complete this manually- see the examples below

In [18]:
print('Well Coordinates for well {}'.format(lab[0].name))
print('x: {}'.format(lab[0].x))
print('y: {}'.format(lab[0].y))
print('z: {}'.format(lab[0].z))

Well Coordinates for well A1
x: 21.81
y: 70.69
z: 7.5


 Let's apply an offset to the labware object and check the coordinates again

In [19]:
lab.offset = (2.0, 3, 3.45)
print('Coordinates after offset')
print('x: {}'.format(lab[0].x))
print('y: {}'.format(lab[0].y))
print('z: {}'.format(lab[0].z))

Coordinates after offset
x: 23.81
y: 73.69
z: 10.95


The offset applied is also saved as a `Labware` attribute too for the user reference 

In [20]:
print('Offset : {}'.format(lab[0].offset))

Offset : (2.0, 3, 3.45)


### 5. Manual Labware Offset Calibration

There may be instances in which the squaring between the deck and the carriage arm of your Jubilee might not be 100% the same. 
To avoid issues with your labware positionining, we've implemented another offset calibration method. 

After placing your labware onto the deck slot, you can follow follow the next few steps to record and apply a namual offset for your labware in *that* slot.
For this you will record the **true** `x` and `y` coordinates 3 corner wells of your labware. Generally these will be you top left, top right, and bottom right wells. For examples for a 96-well wellplate, these will be : `A1`, `A12`, and `H12`, respectively. 

Note: you can perform the next steps using the Camera/WebCamera tool if you want, the procedure is the same. 

1. Pick up a tool (e.g., an OT2 pipette)
2. Navigate to your labware **top_left** well
3. Use the Duet GUI to move the pipette to the "actual" center of your well if the tool is nto perfectly centered over the well. 
4. Record your new `x` and `y` coordinates as a tuple.
5. Repeat steps 2-4 for the **top_right** and **bottom_right** well

After this, you can run the `your_labware.manual_offset()` method. This allows also to save these offsets in the labware `.json` file for future use, as well as later on load them back up without having to re-run this procedure. Note: this will need to be repeated if you laod your labware on a different slot *OR* if you take down and re-build your machine.  

In [21]:
lab.slot = 0  # this will be taken care of by the deck.load_labware() method
UL = (29.3,78.1)
UR = (128.0, 77.9)
BR = (128.2, 15.3)

lab.manual_offset([UL, UR, BR], save= False)

New manual offset applied to 20mlscintillation_12_wellplate_18000ul


You can check if the offset was applied by checking the attribute `manualOffset`.

In [24]:
lab.manualOffset

{'0': [(29.3, 78.1), (128.0, 77.9), (128.2, 15.3)]}